In [1]:
import findspark
findspark.init('/home/gani/spark-2.4.4-bin-hadoop2.7')

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('LogReg').getOrCreate()

In [4]:
from pyspark.ml.classification import LogisticRegression

In [5]:
df = spark.read.format('libsvm').load('sample_libsvm_data.txt')

In [6]:
df.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [7]:
logRegModel = LogisticRegression() 

In [8]:
fittedLogRegModel = logRegModel.fit(df)

In [10]:
summaryLogReg = fittedLogRegModel.summary

In [13]:
summaryLogReg.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [14]:
summaryLogReg.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[19.8534775947478...|[0.99999999761359...|       0.0|
|  1.0|(692,[158,159,160...|[-20.377398194908...|[1.41321555111056...|       1.0|
|  1.0|(692,[124,125,126...|[-27.401459284891...|[1.25804865126979...|       1.0|
|  1.0|(692,[152,153,154...|[-18.862741612668...|[6.42710509170303...|       1.0|
|  1.0|(692,[151,152,153...|[-20.483011833009...|[1.27157209200604...|       1.0|
|  0.0|(692,[129,130,131...|[19.8506078990277...|[0.99999999760673...|       0.0|
|  1.0|(692,[158,159,160...|[-20.337256674833...|[1.47109814695581...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.595579753418...|[3.08850168102631...|       1.0|
|  0.0|(692,[154,155,156...|[19.2708803215613...|[0.99999999572670...|       0.0|
|  0.0|(692,[127

In [15]:
trainDF, testDF = df.randomSplit([0.7,0.3])

In [16]:
finalModel = LogisticRegression()

In [17]:
fitFinal = finalModel.fit(trainDF)

In [18]:
predictNlabel = fitFinal.evaluate(testDF)

In [25]:
predictNlabel.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[123,124,125...|[20.7079480238652...|[0.99999999898456...|       0.0|
|  0.0|(692,[124,125,126...|[29.7596832414010...|[0.99999999999988...|       0.0|
|  0.0|(692,[124,125,126...|[15.9265449815014...|[0.99999987888739...|       0.0|
|  0.0|(692,[124,125,126...|[25.1951101053835...|[0.99999999998857...|       0.0|
|  0.0|(692,[126,127,128...|[22.4566210814015...|[0.99999999982330...|       0.0|
|  0.0|(692,[127,128,129...|[24.1967714749324...|[0.99999999996899...|       0.0|
|  0.0|(692,[151,152,153...|[26.7335428764187...|[0.99999999999754...|       0.0|
|  0.0|(692,[152,153,154...|[21.0531895788117...|[0.99999999928102...|       0.0|
|  0.0|(692,[152,153,154...|[14.2475129903137...|[0.99999935079262...|       0.0|
|  0.0|(692,[153

In [26]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

In [29]:
binEval = BinaryClassificationEvaluator()

In [30]:
finalROC = binEval.evaluate(predictNlabel.predictions)

In [31]:
finalROC

1.0